<a href="https://colab.research.google.com/github/lphansen/RiskUncertaintyValue/blob/main/quickguide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook 
- provides a quick guide to solve the DSGE model using the expansion suite in Section 1, as well as how to compute IRF, approximate, and simulate variables.
- provides some examples for `LinQuadVar` computations in Section 2.

In [16]:
import os
import sys
workdir = os.getcwd()
# !git clone https://github.com/lphansen/RiskUncertaintyValue # Uncomment this when running on Google Colab
# workdir = os.getcwd() + '/RiskUncertaintyValue'             # Uncomment this when running on Google Colab
sys.path.insert(0, workdir+'/src')
import numpy as np
import autograd.numpy as anp
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=200)
from IPython.display import display, HTML
from BY_example_sol import disp
display(HTML("<style>.container { width:97% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

from lin_quad import LinQuadVar
from lin_quad_util import E, concat, next_period, cal_E_ww, lq_sum, simulate
from uncertain_expansion import uncertain_expansion, approximate_fun
np.set_printoptions(suppress=True)

# 1. Five-minute guide to solve the DSGE model using the Expansion Suite

Suppose we would like to solve a modified version of Bansal Yaron Long-run risk model. Its equilibrium condtions include
- One forward-looking condition
$$
\begin{aligned}
\text{pd}_t&=\mathbb{E}[\frac{S_{t+1}}{S_t}(\text{pd}_{t+1}g_{d,t+1}+g_{d,t+1})]
\end{aligned}
$$

- Two state-evolution equation
$$
\begin{aligned}
x_{t+1} & =\alpha x_t+\phi_e \sigma_t w_{1,t+1} \\
\sigma_{t+1}^2= & \sigma^2+v_1\left(\sigma_t^2-\sigma^2\right)+\sigma_w w_{2,t+1}
\end{aligned}
$$

where we have
$$
\begin{aligned}
g_{c,t+1} & =\mu_c+x_t+\phi_c \sigma_t w_{3,t+1} =\log{\frac{C_{t+1}}{C_t}}\\
g_{d,t+1}& =\mu_d+\phi x_t+\pi w_{3,t+1} + \phi_d \sigma_t w_{4,t+1} \\
\frac{S_{t+1}}{S_t}& = \beta \left(\frac{V_{t+1}}{R_t}\right)^{1-\gamma} \left(\frac{V_{t+1}}{R_t}\right)^{\rho-1} \left(\frac{C_{t+1}}{C_t}\right)^{-\rho}
\end{aligned}
$$

The model has one jump variable (In the expansion suite, $J_t$ is used to represent jump variables):
- $\text{pd}_t$ : Price Dividend Ratio

Two tate variables (In the expansion suite, $X_t$ is used to represent state variables):
- $x_{t+1}$ : Growth process
- $\sigma_{t+1}$ : Volatility process

#### The Expansion Suite only takes **five** necessary inputs to solve the DSGE model.

## 1.1 Equilibrium Condition Function

The Equilibrium Condition Function should take the form of the next cell. 

There are several points which we should pay attention to:

1. At the beginning of this function, we need to specify the parameters and variables used in the equilibrium conditions. 

    - The first components of `Var_t` and `Var_tp1` are fixed as `q_t` or `q_tp1`. (See Exploring Recursive Utility Appendix for details.)
    
    - State variables need to follow jump variable.

2. Then we need to specify each equilibrium condition using above specified parameter and variables. 

    - The algorithm requires us to rewrite the equilibrium conditions in the form as equation (1). We need to find what `psi1` is in our equilibrium conditions, specify what  `psi1` look like, and then return it using `psi1` mode. 
    
    - The log change of measure is skipped in the specification of equilibrium conditions.

3. By default (without `mode` specification), the function needs to return the LHS of each equilibrium condition before taking expectation. 

    - The state-evolution equation needs to follow the forward-looking conditions.

$$
\begin{aligned}
\mathbb{E}[{N_{t+1}^*Q_{t+1}^* \psi_1(X_t, J_t, X_{t+1}, J_{t+1})}|{\mathfrak{A}_t}] - \psi_2(X_t, J_t)  &= 0 \\
  \phi\left(X_t, X_{t+1}, J_t, {\sf q}W_{t+1}, {\sf q} \right) &= 0 
\end{aligned}\tag{1}
$$



In [2]:
def eq_cond_BY(Var_t, Var_tp1, W_tp1, q, mode, *args):

    # Parameters for the model
    γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π = args

    # Variables:
    q_t, pd_t, x_t, σ_t_squared = Var_t.ravel()
    q_tp1, pd_tp1, x_tp1, σ_tp1_squared = Var_tp1.ravel()
    w1_tp1, w2_tp1, w3_tp1, w4_tp1 = W_tp1.ravel()

    # Intermedite varibles that facilitates computation
    σ_t = anp.sqrt(σ_t_squared)
    gc_tp1 = μ + x_t + ϕ_c*σ_t*w3_tp1
    gd_tp1 = μ_d + ϕ*x_t + π*σ_t*w3_tp1 + ϕ_d*σ_t*w4_tp1
    
    ## Forward-looking conditions
    psi1_1 = anp.exp(anp.log(β)- ρ*gc_tp1 + gd_tp1)*(anp.exp(pd_tp1) + 1)
    psi2_1 = anp.exp(pd_t)

    # State process
    phi_1 = α * x_t + ϕ_e * σ_t * w1_tp1 - x_tp1
    phi_2 = σ_squared + ν_1 * (σ_t_squared - σ_squared) + σ_w * w2_tp1 - σ_tp1_squared
    
    if mode == 'psi1':
        return np.array([psi1_1])

    return anp.array([psi1_1*anp.exp(q_tp1) - psi2_1, phi_1, phi_2])

## 1.2 Model Steady State Function 

This function takes model parameters as input and outputs the steady state for each variable following the variable order specified in `Var_t` and `Var_tp1`.

- The first one will always be 0, since the steady state of `q_t` is 0.

In [3]:
def ss_func_BY(*args):

    # Parameters for the model
    γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π = args

    sdf = anp.exp(anp.log(β) - ρ*μ)

    return np.array([0, np.log((sdf * np.exp(μ_d))/(1-np.exp(μ_d)*sdf)), 0., σ_squared])

## 1.3 Model Variable Dimension Array

Spcify a simple numpy array for (n_J, n_X, n_W). The dimensions for jump variables, state variables, and shock variables in the model.

In [4]:
var_shape = (1, 2, 4)

## 1.4 Model Parameter Calibration Tuple

Specify all the model parameters using a Python tuple. The first three are fixed recursive utility parameters, γ, β, ρ. 

In [5]:
σ_original = 0.0078

γ = 10
β = .998
ρ = 2./3
α = 0.979
ϕ_e = 0.044 * σ_original
σ_squared = 1.0
ν_1 = 0.987
σ_w = 0.23 * 1e-5 / σ_original**2
μ = 0.0015
μ_d = 0.0015
ϕ_c = 1.0 * σ_original
ϕ = 3.0
ϕ_d = 4.5 * σ_original
π = 0.0

args = (γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π)

## 1.5 Log Consumption Growth Function

The expansion suite deals with the recursive utility in equation (2). We need to specify how to compute the log-growth of consumption using the defined variables `Var_t` and `Var_tp1` in the equilibrium conditions. 

- For some models the $C_t$ is replaced with $U_t$, then we need to specify the log growth process for $U_t$. 
- For some models with preference shocks, $C_t = \lambda_t \tilde{C}_t$, we need to specify the log growth process for $\lambda_t \tilde{C}_t$. 

$$
V_t=\left[(1-\beta)\left(C_t\right)^{1-\rho}+\beta\left(R_t\right)^{1-\rho}\right]^{\frac{1}{1-\rho}}\tag{2}
$$

In [6]:
def gc_tp1_approx(Var_t, Var_tp1, W_tp1, q, *args):

    # Parameters for the model
    γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π = args

    # Variables:
    q_t, pd_t, x_t, σ_t_squared = Var_t.ravel()
    q_tp1, pd_tp1, x_tp1, σ_tp1_squared = Var_tp1.ravel()
    w1_tp1, w2_tp1, w3_tp1, w4_tp1 = W_tp1.ravel()
    σ_t = anp.sqrt(σ_t_squared)
    
    gc_tp1 = μ + x_t + ϕ_c*σ_t*w3_tp1
    
    return gc_tp1

## 1.6 Solve the BY model using the Expansion Suite 

The expansion suite takes the above five necessary inputs to solve a model. The solution is stored into a class named `ModelSolution`(Please refer to uncertainexpansion.ipynb for details.) under the Linear Quadratic Framework. The computation of the framework is built on a `LinQuadVar` class.

In [7]:
# Solve BY model
eq = eq_cond_BY
ss = ss_func_BY 
var_shape = (1, 2, 4)
gc_tp1_fun = gc_tp1_approx
args = (γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π)
approach = '1' # See Exploring Recursive Utility Appendix for difference about approach '1' and '2'
ModelSol = uncertain_expansion(eq, ss, var_shape, args, gc_tp1_fun, approach)

Iteration 1: error = 30.6278088
Iteration 2: error = 0


## 1.7 Model Solutions

Below cells display analytical forms of the approximation results for jump variables and state variables. These variables are all stored as `LinQuadVar`. 

In [8]:
## PD ratio approximation results, shown in the LinQuadVar coefficients form
ModelSol['JX_t'][0].coeffs

{'xx': array([[180.11784618,   0.        ,   0.        ,   0.        ]]),
 'c': array([[8.90178722]]),
 'x': array([[84.54940357, -0.28169922]]),
 'x2': array([[51.92256612,  0.        ]])}

In [9]:
## PD ratio approximation results, shown in the Latex analytical form
disp(ModelSol['JX_t'][0],'\\text{pd}_t') 

<IPython.core.display.Math object>

In [29]:
## PD ratio zeroth order approximation results
disp(ModelSol['J0_t'][0],'\\text{pd}^0_t')

<IPython.core.display.Math object>

In [30]:
## PD ratio first order approximation results
disp(ModelSol['J1_t'][0],'\\text{pd}^1_t')

<IPython.core.display.Math object>

In [31]:
## PD ratio second order approximation results
disp(ModelSol['J2_t'][0],'\\text{pd}^2_t')

<IPython.core.display.Math object>

In [36]:
## Growth process first order approximation results
disp(ModelSol['X1_tp1'][0],'\\text{x}^1_{t+1}') 

<IPython.core.display.Math object>

In [38]:
## Growth process second order approximation results
disp(ModelSol['X2_tp1'][0],'\\text{x}^2_{t+1}') 

<IPython.core.display.Math object>

In [42]:
## Volatility process first order approximation results
disp(ModelSol['X1_tp1'][1],'{\\sigma^2_{t+1}}^1') 

<IPython.core.display.Math object>

In [44]:
## Volatility process second order approximation results
disp(ModelSol['X2_tp1'][1],'{\\sigma^2_{t+1}}^2') 

<IPython.core.display.Math object>

## 1.8 Approximate Variables

If some variables can be computed using solved model variables, we can write a function to specify how to approximate, and then use `approximate_fun` to compute the approximation of these variables using model solutions. The below example computes the approximation of log dividend growth process.

In [15]:
def gd_tp1_approx(Var_t, Var_tp1, W_tp1, q, *args):

    # Parameters for the model
    γ, β, ρ, α, ϕ_e, σ_squared, ν_1, σ_w, μ, μ_d, ϕ, ϕ_d, ϕ_c, π = args

    # Variables:
    pd_t, x_t, σ_t_squared = Var_t.ravel()
    pd_tp1, x_tp1, σ_tp1_squared = Var_tp1.ravel()
    w1_tp1, w2_tp1, w3_tp1, w4_tp1 = W_tp1.ravel()
    σ_t = anp.sqrt(σ_t_squared)
    
    gd_tp1 = μ_d + ϕ*x_t + π*σ_t*w3_tp1 + ϕ_d*σ_t*w4_tp1
    
    return gd_tp1

n_J, n_X, n_W = var_shape
gd_tp1_list = approximate_fun(gd_tp1_approx, ModelSol['ss'], (1, n_X, n_W), ModelSol['JX1_t'], ModelSol['JX2_t'], ModelSol['X1_tp1'], ModelSol['X2_tp1'], args)
gd_tp1 = {'gd_tp1':gd_tp1_list[0],\
        'gd0_tp1':gd_tp1_list[1],\
        'gd1_tp1':gd_tp1_list[2],\
        'gd2_tp1':gd_tp1_list[3]}
gd_tp1['gd1_tp1'].coeffs

{'x': array([[3., 0.]]), 'w': array([[0.    , 0.    , 0.    , 0.0351]])}

## 1.9 Model Simulations
Given a series of shock processes, we can simulate variables using the `simulate` function. Below examples simulate the growth of consumption and growth process with 400 periods i.i.d standard multivariate normal shocks.

In [24]:
Ws = np.random.multivariate_normal(np.zeros(n_W),np.eye(n_W),size = 400)
gc_tp1_sim = simulate(ModelSol['gc_tp1'], ModelSol['X1_tp1'], ModelSol['X2_tp1'],Ws)
x_tp1_sim = simulate(ModelSol['JX_tp1'][0], ModelSol['X1_tp1'], ModelSol['X2_tp1'],Ws)

## ModelSolution also has an attribute to simulate all jump and state varibles 
JX_sim = ModelSol.simulate(Ws)

## 1.10 IRF

`ModelSolution` has a method `IRF` to compute the IRF of all jump and state variables. Below we calculate the IRF of all variables with respect to the first shock for 400 periods.

In [23]:
states, controls = ModelSol.IRF(400, 0)

# 2 Examples to use LinQuadVar in Computation

## 2.1 Define a LinQuadVar

Below we extract the steady states for the state evolution processes from the previous model solution, and construct it as a 2-dimension LinQuadVar.

In [53]:
n_J, n_X, n_W = ModelSol['var_shape']
X0_tp1 = LinQuadVar({'c':np.array([[ModelSol['ss'][1]],[ModelSol['ss'][2]]])}, shape = (2, n_X, n_W))
X0_tp1.coeffs

{'c': array([[0.],
        [1.]])}

## 2.2 LinQuad Operations
Below show two ways to add two LinQuads, we use the zeroth order, first order, second order approximation results to construct the approximation for the growth process or both state evolution processes.

In [68]:
x_tp1 = X0_tp1[0] + ModelSol['X1_tp1'][0]  + 0.5 * ModelSol['X2_tp1'][0] 
disp(x_tp1,'\\text{x}_{t+1}') 

<IPython.core.display.Math object>

In [69]:
lq_sum([X0_tp1, ModelSol['X1_tp1'], 0.5 * ModelSol['X2_tp1']]).coeffs

{'x': array([[0.979, 0.   ],
        [0.   , 0.987]]),
 'xw': array([[0.       , 0.       , 0.       , 0.       , 0.0001716, 0.       ,
         0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       ]]),
 'w': array([[0.0003432 , 0.        , 0.        , 0.        ],
        [0.        , 0.03780408, 0.        , 0.        ]]),
 'x2': array([[0.4895, 0.    ],
        [0.    , 0.4935]]),
 'c': array([[0.],
        [1.]])}

## 2.3 LinQuad Spilt and Concat
`Spilt` breaks multiple dimension LinQuad into one-dimension LinQuads, while `Concat` does the inverse operation

In [73]:
x1_tp1, sigma1_tp1 = ModelSol['X1_tp1'].split()
concat([x1_tp1, sigma1_tp1])

## 2.4 Evaluate a LinQuad
Given specific values for $X_{t}$, $W_{t+1}$, we can evaluate a `LinQuadVar`. Below evaluate the PD ratio, growth process, and volatility process.

In [81]:
x1 = np.zeros([n_X ,1])
x2 = np.zeros([n_X ,1])
w = np.random.multivariate_normal(np.zeros(n_W),np.eye(n_W),size = 1).T
ModelSol['JX_tp1'](*(x1,x2,w))

array([[ 8.87618617],
       [-0.00031046],
       [ 0.99775924]])

## 2.5 Adjust Conditional Information Time 
Since some LinQuadVars (e.g. solved jump variables) are expressed as a function of time $t$ state variables without shocks, when we would like to compute its next period expression, $t+1$, with shocks, but still as a function of time $t$ state variables, we can use function `next_period`.

In [83]:
pd1_tp1 = next_period(ModelSol['J1_t'], ModelSol['X1_tp1'])
disp(pd1_tp1, '\\text{pd}^1_{t+1}')

<IPython.core.display.Math object>

In [84]:
pd2_tp1 = next_period(ModelSol['J2_t'], ModelSol['X1_tp1'], ModelSol['X2_tp1'])
disp(pd2_tp1, '\\text{pd}^2_{t+1}')

<IPython.core.display.Math object>

## 2.6 Compute the Expectation of time t+1 LinQuadVar

Suppose the distribution of shock has a constant mean and variance (not state dependent), we can use `E` to compute the expectation of time t+1 LinQuadVar.


In [89]:
E_w = np.zeros([n_W,1])
cov_w = np.eye(n_W)
E_ww = cal_E_ww(E_w, cov_w)
E_pd1_tp1 = E(pd1_tp1, E_w, E_ww)
disp(E_pd1_tp1, '\mathbb{E}[\\text{pd}^1_{t+1}|\mathfrak{F_t}]')

<IPython.core.display.Math object>